In [3]:
import pandas as pd
import numpy as np

In [13]:
import nltk

In [14]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import PorterStemmer

In [15]:
import matplotlib.pyplot as plt

Separo el texto en palabras y elimino las stop words. 
No aplico streamming para quedarme con las palabras completas.

In [16]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/cecilia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/cecilia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
train = pd.read_csv('Archivos/entrenamiento_df.csv')
test = pd.read_csv('Archivos/test_df.csv')

In [19]:
# Separo el texto en palabras
# Elimino las stop words y paso las palabras a minusculas
train['text'] = train['text'].apply(lambda x: RegexpTokenizer(r'\w+').tokenize(x))
train['text'] = train['text'].apply(lambda x: [w.lower() for w in x if w not in stopwords.words('english')])

In [20]:
# Hago lo mismo con el archivo test
test['text'] = test['text'].apply(lambda x: RegexpTokenizer(r'\w+').tokenize(x))
test['text'] = test['text'].apply(lambda x: [w.lower() for w in x if w not in stopwords.words('english')])

### Creo nuevos features mediante vectores de texto ya entrenados

In [11]:
import gensim.downloader as api

In [12]:
# load pre-trained word-vectors from gensim-data
word_vectors = api.load("glove-wiki-gigaword-100") 

In [11]:
train['text'][0]

['our', 'deeds', 'reason', 'earthquake', 'may', 'allah', 'forgive', 'us']

In [271]:
word_vectors.most_similar(positive=train['text'][0])

[('we', 0.8172246217727661),
 ('not', 0.8154860734939575),
 ('what', 0.8063154816627502),
 ('do', 0.8049666285514832),
 ('...', 0.8043433427810669),
 ('know', 0.8036221861839294),
 ('come', 0.7973394989967346),
 ('if', 0.7953765988349915),
 ('why', 0.7918872237205505),
 ('believe', 0.7906121015548706)]

In [272]:
word_vectors.most_similar(negative=train['text'][0])

[('purva.patel@chron.com', 0.698187530040741),
 ('tom.fowler@chron.com', 0.6924424767494202),
 ('brett.clanton@chron.com', 0.6868560910224915),
 ('spodumene', 0.6839196085929871),
 ('atentamente', 0.6803922653198242),
 ('___________________________________________________________',
  0.6781449317932129),
 ('rungfapaisarn', 0.6756051778793335),
 ('ryryryryryry', 0.675279438495636),
 ('jenalia.moreno@chron.com', 0.6748037934303284),
 ('kraz', 0.6747796535491943)]

In [274]:
word_vectors.doesnt_match(train['text'][0])

'earthquake'

In [269]:
train['text'][1]

['forest', 'fire', 'near', 'la', 'ronge', 'sask', 'canada']

In [268]:
word_vectors.most_similar(positive=train['text'][1])

[('northern', 0.6509563326835632),
 ('southern', 0.6403713226318359),
 ('northeast', 0.6403039693832397),
 ('park', 0.6400458812713623),
 ('mountain', 0.6395445466041565),
 ('southwest', 0.6368558406829834),
 ('river', 0.6357375383377075),
 ('located', 0.6302077174186707),
 ('northwest', 0.6292986273765564),
 ('area', 0.6256020069122314)]

In [23]:
# Elimino las palabras del texto que no estan en el vocabulario
def vocabulario(palabras):
    lista = []
    for w in palabras:
        if w in word_vectors.vocab:
            lista.append(w) 
    if (len(lista) == 0):
        return ['nothing']
    return lista

In [13]:
similares = train['text'].agg(lambda x: word_vectors.most_similar(positive=vocabulario(x), topn=1))

In [14]:
similares

0             [(we, 0.8172246217727661)]
1       [(northern, 0.6509563326835632)]
2           [(they, 0.8379936218261719)]
3             [(10, 0.7982413172721863)]
4            [(out, 0.7865819334983826)]
                      ...               
7608     [(building, 0.754969596862793)]
7609        [(this, 0.8649874329566956)]
7610          [(03, 0.6322605013847351)]
7611      [(another, 0.787148118019104)]
7612        [(last, 0.7914666533470154)]
Name: text, Length: 7613, dtype: object

In [15]:
palabras_similares = [x[0][0] for x in similares]
semejanzas = [x[0][1] for x in similares]

In [16]:
palabras_similares[:5]

['we', 'northern', 'they', '10', 'out']

In [17]:
semejanzas[0:5]

[0.8172246217727661,
 0.6509563326835632,
 0.8379936218261719,
 0.7982413172721863,
 0.7865819334983826]

In [136]:
sim_train = pd.DataFrame({'target':train['target'], 'semejanza':semejanzas})
sim_train.to_csv('Archivos/train_semejanzas.csv', index=False)

In [137]:
sim_train.head()

,target,semejanza
0,1,0.817225
1,1,0.650956
2,1,0.837994
3,1,0.798241
4,1,0.786582


In [138]:
train_palabras = pd.DataFrame({'target':train['target'], 'palabra_similar':palabras_similares})
train_palabras['palabra_similar'].value_counts()

so           414
one          405
you          378
just         363
this         295
            ... 
landfall       1
fabric         1
group          1
lane           1
agreement      1
Name: palabra_similar, Length: 937, dtype: int64

In [139]:
# Agrego feature con el mean encoding de las palabras similares
mean_palabra_similar = train_palabras.groupby('palabra_similar')['target'].agg(['mean','count'])
mean_palabra_similar.count()

mean     937
count    937
dtype: int64

In [140]:
# Dejo solo los promedios de las palabras con mas de 5 ocurrencias
mean_palabra_similar = mean_palabra_similar[mean_palabra_similar['count'] > 5].drop(['count'],axis=1)
mean_palabra_similar.count()

mean    173
dtype: int64

Son pocos registros como para crear otro feature

In [102]:
# Encuentro las palabras que menos se acercan a los textos
diferentes = train['text'].agg(lambda x: word_vectors.most_similar(negative=vocabulario(x), topn=1))

In [105]:
diferentes

0            [(purva.patel@chron.com, 0.698187530040741)]
1                          [(yassen, 0.5855438113212585)]
2                            [(zety, 0.7851395606994629)]
3                   [(rungfapaisarn, 0.7050082087516785)]
4       [(____________________________________________...
                              ...                        
7608                          [(zety, 0.750871479511261)]
7609                         [(zety, 0.8317862749099731)]
7610                         [(rw95, 0.6121312379837036)]
7611    [(____________________________________________...
7612                         [(zety, 0.8025525212287903)]
Name: text, Length: 7613, dtype: object

In [141]:
palabras_diferentes = [x[0][0] for x in diferentes]
diferencias = [x[0][1] for x in diferentes]

In [142]:
palabras_diferentes[:5]

['purva.patel@chron.com',
 'yassen',
 'zety',
 'rungfapaisarn',
 '___________________________________________________________']

In [143]:
diferencias[0:5]

[0.698187530040741,
 0.5855438113212585,
 0.7851395606994629,
 0.7050082087516785,
 0.7276370525360107]

In [144]:
sim_train['diferencia'] = diferencias

In [145]:
sim_train.head()

,target,semejanza,diferencia
0,1,0.817225,0.698188
1,1,0.650956,0.585544
2,1,0.837994,0.785140
3,1,0.798241,0.705008
4,1,0.786582,0.727637


In [146]:
sim_train.to_csv('Archivos/train_semejanzas.csv', index=False)

In [147]:
train_palabras['palabra_diferente'] = palabras_diferentes
train_palabras['palabra_diferente'].value_counts()

___________________________________________________________    1893
zety                                                           1812
brett.clanton@chron.com                                         574
ryryryryryry                                                    285
tom.fowler@chron.com                                            217
                                                               ... 
ojcl                                                              1
ukita                                                             1
zongmi                                                            1
katzenbach                                                        1
wattanachai                                                       1
Name: palabra_diferente, Length: 1158, dtype: int64

In [148]:
# Agrego feature con el mean encoding de las palabras diferentes
mean_palabra_dif = train_palabras.groupby('palabra_diferente')['target'].agg(['mean','count'])
mean_palabra_dif.count()

mean     1158
count    1158
dtype: int64

In [149]:
mean_palabra_dif['count'].mean()

6.57426597582038

In [150]:
# Dejo solo los promedios de las palabras con mas de 5 ocurrencias
mean_palabra_dif = mean_palabra_dif[mean_palabra_dif['count'] > 5].drop(['count'],axis=1)
mean_palabra_dif.count()

mean    75
dtype: int64

Son pocos registros como para crear otro feature

In [119]:
# Encuentro la palabra que no coincide con el resto del texto
infiltrados = train['text'].agg(lambda x: word_vectors.doesnt_match(vocabulario(x)))

/home/cecilia/.local/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [120]:
infiltrados

0       earthquake
1             sask
2         notified
3        wildfires
4            pours
           ...    
7608          http
7609     troubling
7610           5km
7611      collided
7612          http
Name: text, Length: 7613, dtype: object

In [151]:
train_palabras['palabra_infiltrada'] = infiltrados
train_palabras['palabra_infiltrada'].value_counts()

http         1619
https         306
û              89
lol            47
rt             46
             ... 
sabotage        1
ben             1
charminar       1
hostages        1
pharma          1
Name: palabra_infiltrada, Length: 3171, dtype: int64

In [152]:
train_palabras.head()

,target,palabra_similar,palabra_diferente,palabra_infiltrada
0,1,we,purva.patel@chron.com,earthquake
1,1,northern,yassen,sask
2,1,they,zety,notified
3,1,10,rungfapaisarn,wildfires
4,1,out,______________________________________________...,pours


In [153]:
# Agrego feature con el mean encoding de las palabras infiltradas
mean_infiltradas = train_palabras.groupby('palabra_infiltrada')['target'].agg(['mean','count'])
mean_infiltradas.count()

mean     3171
count    3171
dtype: int64

In [154]:
mean_infiltradas['count'].mean()

2.400819930621255

In [155]:
# Dejo solo los promedios de las palabras con mas de 5 ocurrencias
mean_infiltradas = mean_infiltradas[mean_infiltradas['count'] > 5].drop(['count'],axis=1)
mean_infiltradas.count()

mean    141
dtype: int64

Son pocos registros como para crear otro feature

In [156]:
sim_train.head()

,target,semejanza,diferencia
0,1,0.817225,0.698188
1,1,0.650956,0.585544
2,1,0.837994,0.785140
3,1,0.798241,0.705008
4,1,0.786582,0.727637


In [157]:
sim_train.to_csv('Archivos/train_semejanzas.csv', index=False)

In [24]:
# Calculo la similaridad entre el texto y la palabra infiltrada
sim_train['simil_intruso'] = train['text'].agg(lambda x: word_vectors.n_similarity(vocabulario(x), word_vectors.doesnt_match(vocabulario(x))))

/home/cecilia/.local/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [25]:
sim_train.head()

,target,semejanza,diferencia,simil_intruso
0,1,0.817225,0.698188,0.515924
1,1,0.650956,0.585544,0.430637
2,1,0.837994,0.785140,0.503261
3,1,0.798241,0.705008,0.432212
4,1,0.786582,0.727637,0.378606


In [26]:
sim_train.to_csv('Archivos/train_semejanzas.csv', index=False)

#### Repito el mecanismo para el archivo de test

In [158]:
similares_test = test['text'].agg(lambda x: word_vectors.most_similar(positive=vocabulario(x), topn=1))

In [159]:
similares_test

0          [(accident, 0.7852973937988281)]
1               [(come, 0.845439612865448)]
2              [(away, 0.8163744211196899)]
3          [(wildfire, 0.6221873164176941)]
4          [(mainland, 0.6732683777809143)]
                       ...                 
3258         [(center, 0.6763065457344055)]
3259           [(just, 0.8925397396087646)]
3260            [(new, 0.7113810181617737)]
3261    [(environment, 0.7011711597442627)]
3262       [(approved, 0.7105252146720886)]
Name: text, Length: 3263, dtype: object

In [160]:
palabras_similares = [x[0][0] for x in similares_test]
semejanzas = [x[0][1] for x in similares_test]

In [161]:
palabras_similares[:5]

['accident', 'come', 'away', 'wildfire', 'mainland']

In [162]:
semejanzas[0:5]

[0.7852973937988281,
 0.845439612865448,
 0.8163744211196899,
 0.6221873164176941,
 0.6732683777809143]

In [163]:
sim_test = pd.DataFrame({'id':test['id'], 'semejanza':semejanzas})
sim_test.to_csv('Archivos/test_semejanzas.csv', index=False)

In [164]:
sim_test.head()

,id,semejanza
0,0,0.785297
1,2,0.845440
2,3,0.816374
3,9,0.622187
4,11,0.673268


In [165]:
test_palabras = pd.DataFrame({'id':test['id'], 'palabra_similar':palabras_similares})
test_palabras['palabra_similar'].value_counts()

so           180
just         169
one          154
you          152
this         137
            ... 
displaced      1
2008           1
refugees       1
2007           1
insurance      1
Name: palabra_similar, Length: 593, dtype: int64

In [166]:
sim_test.head()

,id,semejanza
0,0,0.785297
1,2,0.845440
2,3,0.816374
3,9,0.622187
4,11,0.673268


In [167]:
# Encuentro las palabras que menos se acercan a los textos
diferentes_test = test['text'].agg(lambda x: word_vectors.most_similar(negative=vocabulario(x), topn=1))

In [168]:
diferentes_test

0       [(____________________________________________...
1                            [(zety, 0.7198097109794617)]
2                            [(zety, 0.7555083632469177)]
3                           [(30.11, 0.4841735363006592)]
4                          [(hoeyer, 0.5357732176780701)]
                              ...                        
3258                       [(3.6730, 0.6118791103363037)]
3259    [(____________________________________________...
3260    [(____________________________________________...
3261                 [(kallicharran, 0.6488111615180969)]
3262                    [(ambareesh, 0.6341420412063599)]
Name: text, Length: 3263, dtype: object

In [169]:
palabras_diferentes_test = [x[0][0] for x in diferentes_test]
diferencias_test = [x[0][1] for x in diferentes_test]

In [170]:
palabras_diferentes_test[:5]

['___________________________________________________________',
 'zety',
 'zety',
 '30.11',
 'hoeyer']

In [171]:
diferencias_test[0:5]

[0.6520140767097473,
 0.7198097109794617,
 0.7555083632469177,
 0.4841735363006592,
 0.5357732176780701]

In [172]:
sim_test['diferencia'] = diferencias_test

In [173]:
sim_test.head()

,id,semejanza,diferencia
0,0,0.785297,0.652014
1,2,0.845440,0.719810
2,3,0.816374,0.755508
3,9,0.622187,0.484174
4,11,0.673268,0.535773


In [174]:
sim_test.to_csv('Archivos/test_semejanzas.csv', index=False)

In [175]:
test_palabras['palabra_diferente'] = palabras_diferentes_test
test_palabras['palabra_diferente'].value_counts()

zety                                                           821
___________________________________________________________    766
brett.clanton@chron.com                                        235
tom.fowler@chron.com                                           125
ryryryryryry                                                   118
                                                              ... 
metemma                                                          1
huckins                                                          1
liepa                                                            1
gédéon                                                           1
useni                                                            1
Name: palabra_diferente, Length: 601, dtype: int64

In [176]:
sim_test.head()

,id,semejanza,diferencia
0,0,0.785297,0.652014
1,2,0.845440,0.719810
2,3,0.816374,0.755508
3,9,0.622187,0.484174
4,11,0.673268,0.535773


In [177]:
# Encuentro la palabra que no coincide con el resto del texto
infiltrados_test = test['text'].agg(lambda x: word_vectors.doesnt_match(vocabulario(x)))

/home/cecilia/.local/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [178]:
infiltrados_test

0         terrible
1       earthquake
2            geese
3          spokane
4         soudelor
           ...    
3258     fasteners
3259            ri
3260    derailment
3261           meg
3262     activated
Name: text, Length: 3263, dtype: object

In [179]:
test_palabras['palabra_infiltrada'] = infiltrados_test
test_palabras['palabra_infiltrada'].value_counts()

http       659
https      155
û           33
lol         22
rt          20
          ... 
attila       1
dominos      1
sens         1
harry        1
fire         1
Name: palabra_infiltrada, Length: 1584, dtype: int64

In [29]:
sim_test.to_csv('Archivos/test_semejanzas.csv', index=False)

In [27]:
sim_test = pd.read_csv('Archivos/test_semejanzas.csv')

In [28]:
# Calculo la similaridad entre el texto y la palabra infiltrada
sim_test['simil_intruso'] = test['text'].agg(lambda x: word_vectors.n_similarity(vocabulario(x), word_vectors.doesnt_match(vocabulario(x))))

/home/cecilia/.local/lib/python3.6/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


In [29]:
sim_test.head()

,id,semejanza,diferencia,simil_intruso
0,0,0.785297,0.652014,0.418288
1,2,0.845440,0.719810,0.513763
2,3,0.816374,0.755508,0.347620
3,9,0.622187,0.484174,0.071788
4,11,0.673268,0.535773,0.259028


In [30]:
sim_test.to_csv('Archivos/test_semejanzas.csv', index=False)